In [1]:
import pandas as pd
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")
#pd.set_option('display.max_rows', None)

# Load the Excel file to examine its structure and content
file_path = "C:\\Users\\arcan\\OneDrive\\Ambiente de Trabalho\\My apps\\python\\database project\\Billionaires.xlsx"
excel_data = pd.ExcelFile(file_path)
# Load the content of the 'Billionaires' sheet to understand its structure
billionaires_data = excel_data.parse('Billionaires')
versatyle_dictionary = {}
versatyle_counter = 1

In [2]:
# Prepare the Continents table
continents = billionaires_data[['continent']].drop_duplicates()

continents['continent_id'] = range(1, len(continents) + 1)

continents_final = continents[['continent_id', 'continent']]

In [3]:
# Prepare the countries table
countries = billionaires_data[['country_of_residence', 'continent', 'country_lat', 
                               'country_long', 'country_pop', 'life_expectancy', 'cpi_country', 'cpi_change_country', 'g_tertiary_ed_enroll', 'g_primary_ed_enroll', 
                               'tax_revenue', 'tax_rate', 'gdp_country']].drop_duplicates()

countries = countries.merge(continents_final[['continent_id', 'continent']], on='continent', how='left')
countries['country_id'] = range(1, len(countries) + 1)

countries_final = countries[['country_id', 'country_of_residence', 'continent_id', 'country_lat', 
                               'country_long', 'country_pop', 'life_expectancy', 'cpi_country', 'cpi_change_country', 'gdp_country', 'g_tertiary_ed_enroll', 'g_primary_ed_enroll', 
                               'tax_revenue', 'tax_rate']]

countries_final.rename(columns={"country_lat": "country_latitude"}, inplace=True)
countries_final.rename(columns={"country_long": "country_longitude"}, inplace=True)
countries_final.rename(columns={"life_expectancy": "life_exp"}, inplace=True)
countries_final.rename(columns={"cpi_change_country": "cpi_change"}, inplace=True)
countries_final.rename(columns={"g_tertiary_ed_enroll": "g_tertiary"}, inplace=True)
countries_final.rename(columns={"g_primary_ed_enroll": "g_primary"}, inplace=True)

In [4]:
# Prepare the companies table
companies = billionaires_data[['source', 'industry']].drop_duplicates()

companies['company_id'] = range(1, len(companies) + 1)

companies_final = companies[['company_id', 'source', 'industry']]
companies_final.rename(columns={'source': 'resource'}, inplace=True)

In [5]:
#Preprare the regions table
regions = billionaires_data[['residence_region']].drop_duplicates()

regions['region_id'] = range(1, len(regions) + 1)

regions_final = regions[['region_id', 'residence_region']]

regions_final.rename(columns={"residence_region": "res_region"}, inplace=True)

In [6]:
# Prepare the states table
states = billionaires_data[['residence_state','residence_region','country_of_residence']].drop_duplicates(['residence_state'])
states.rename(columns={"residence_region": "res_region"}, inplace=True)

states = states.merge(regions_final[['region_id', 'res_region']], on='res_region', how='left')
states = states.merge(countries_final[['country_id', 'country_of_residence']], on='country_of_residence', how='left')

states['state_id'] = range(1, len(states) + 1)

states_final = states[['state_id', 'residence_state', 'region_id', 'country_id']]

states_final.rename(columns={"residence_state": "res_state"}, inplace=True)

In [7]:
# Prepare the cities table
cities = billionaires_data[['city_of_residence', 'residence_state', 'residence_region', 'country_of_residence']].drop_duplicates(['city_of_residence', 'residence_state', 'country_of_residence'])
cities = cities.merge(countries_final[['country_of_residence', 'country_id']], on='country_of_residence', how='left')
cities.rename(columns={"residence_state": "res_state"}, inplace=True)
cities = cities.merge(states_final[['res_state', 'state_id']], on='res_state', how='left')
cities['city_id'] = range(1, len(cities) + 1)

cities_final = cities[['city_id', 'city_of_residence', 'state_id', 'country_id']]

In [8]:
# Prepare the billionaires table
billionaires = billionaires_data[['first_name','last_name', 'full_name', 'age', 'gender', 'birth_date', 'birth_day', 'birth_month', 'birth_year', 'position', 'wealth', 'city_of_residence', 'residence_state', 'citizenship', 'residence_region', 'country_of_residence']]
billionaires.rename(columns={"residence_region": "res_region"}, inplace=True)
billionaires.rename(columns={"residence_state": "res_state"}, inplace=True)
billionaires = billionaires.merge(cities[['city_of_residence', 'res_state', 'country_of_residence', 'city_id']], on=['city_of_residence', 'res_state', 'country_of_residence',], how='left')
billionaires = billionaires.merge(countries_final[['country_of_residence', 'country_id']], on='country_of_residence', how='left')
billionaires['billionaire_id'] = range(1, len(billionaires) + 1)

billionaires_final = billionaires[['billionaire_id','first_name','last_name', 'full_name', 'age', 'gender', 'birth_date', 'birth_day', 'birth_month', 'birth_year', 'position', 'wealth', 'citizenship', 'city_id','country_id']]

billionaires_final.rename(columns={"first_name": "firstname"}, inplace=True)
billionaires_final.rename(columns={"last_name": "lastname"}, inplace=True)
billionaires_final.rename(columns={"full_name": "fullname"}, inplace=True)

In [9]:
# Prepare the Billionaire_Companies table
billionaire_companies = billionaires_data[['full_name','gender', 'age', 'source', 'industry']]
billionaire_companies.rename(columns={"full_name": "fullname"}, inplace=True)
billionaire_companies.rename(columns={"source": "resource"}, inplace=True)

billionaire_companies = billionaire_companies.merge(billionaires_final[['fullname', 'gender', 'age', 'billionaire_id']], on=['fullname', 'gender', 'age'], how='left')
billionaire_companies = billionaire_companies.merge(companies_final[['resource', 'industry', 'company_id']], on=['resource', 'industry'], how='left')

billionaire_companies_final = billionaire_companies[['billionaire_id', 'company_id']]

In [11]:
import sqlite3

# Paths to your database file and pandas DataFrames
db_path = "C:\\Users\\arcan\\OneDrive\\Ambiente de Trabalho\\My apps\\python\\database project\\Billionaires.db" 

# DataFrames to insert
dataframes = {
    "Countries": countries_final,
    "Cities": cities_final,
    "Billionaires": billionaires_final,
    "Companies": companies_final,
    "Billionaire_Companies": billionaire_companies_final,
    "States": states_final,
    "Regions": regions_final,
    "Continents": continents_final
}

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

cursor = conn.cursor()
for key in dataframes.keys():
    delete_query = f"DELETE FROM {key};"
    cursor.execute(delete_query)
conn.commit()

# Insert data into the database
for table_name, dataframe in dataframes.items():
    dataframe.to_sql(table_name, conn, if_exists='append', index=False)
    print(f"Data inserted successfully into {table_name} table.")

conn.commit()  # Commit the changes
conn.close()   # Close the database connection

Data inserted successfully into Countries table.
Data inserted successfully into Cities table.
Data inserted successfully into Billionaires table.
Data inserted successfully into Companies table.
Data inserted successfully into Billionaire_Companies table.
Data inserted successfully into States table.
Data inserted successfully into Regions table.
Data inserted successfully into Continents table.
